# 1.1 隨機森林 

資料集

```python
import numpy as np

```

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('bmh')
from tqdm import tqdm
import requests, datetime
from bs4 import BeautifulSoup
